# Extract BARRA/ERA5

In [2]:
import sys
import os
import glob
import xarray as xr
sys.path.append('/g/data/mn51/users/jb6465/code/wetsa-wp3')
from utils import *

import dask
import tempfile
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import warnings
import logging
warnings.filterwarnings('ignore') 
logging.getLogger("distributed.worker.memory").setLevel(logging.ERROR)
logging.getLogger('flox').setLevel(logging.WARNING)

### Extract BARRA (faster without dask)

In [2]:
extracted_data_save_dir = '/g/data/mn51/users/jb6465/data/wetsa-wp3'
time_step = '1hr' #10min 1hr 3hr day fx mon
nation_domain = 'indonesia'

for target_var in ['tas', 'uas', 'vas', 'sfcWind', 'rsds', 'pr']:
    print(target_var)
    for year in list(range(1979,2024)):
      barra_extract(target_var, extracted_data_save_dir, time_step, nation_domain, year)

tas
uas
vas
sfcWind
rsds
pr


### Extract ERA5 (faster with dask)

In [10]:
dask.config.set({'array.chunk-size': "256 MiB",'array.slicing.split_large_chunks': True, 'distributed.comm.timeouts.connect': '120s', 'distributed.comm.timeouts.tcp': '120s', 'distributed.comm.retry.count': 10, 'distributed.scheduler.allowed-failures': 20, "distributed.scheduler.worker-saturation": 1.1})
client = Client(n_workers=12, threads_per_worker=1, local_directory = tempfile.mkdtemp(), memory_limit = "63000mb")

In [11]:
extracted_data_save_dir = '/g/data/mn51/users/jb6465/data/wetsa-wp3'
nation_domain = 'indonesia'

for target_var in ['10u', '10v', 'ssrd', 'tp', '2t']:
    print(target_var)
    for year in list(range(1979,2024)):
      era5_extract(target_var, extracted_data_save_dir, nation_domain, year)  

10u
10v
ssrd
tp
2t


In [13]:
client.close()